# Gradio Interface to Chat 

This is the final product showcasing the CRM AI Assistant built in the Gradio environment. The current version uses DummyCRM, a test module that simulates the interface and functionality for demonstration purposes.

Our CRM AI Assistant can:

* Generate intelligent business insights and pipeline analyses using advanced prompts, providing evaluations of conversion rates, deal performance, and strategic recommendation for sales optimization (TO-DO).

* Produce detailed Matplotlib/Seaborn charts visualizing opportunity pipeline stages, revenue distribution by industry, lead conversion rates, and deal amount distributions across the sales organization.

Advanced features will be implemented soon!

### Imports and Setup

In [16]:
import os
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
import time

# from your_llm_module import CRMAssistant
# from your_search_module import indexer 

In [17]:
# Load clean data
base_path = "data_directory/clean_data"
pipeline = pd.read_csv(os.path.join(base_path, "Pipeline.csv"))
accounts = pd.read_csv(os.path.join(base_path, "Accounts.csv"))
teams = pd.read_csv(os.path.join(base_path, "Teams.csv"))
products = pd.read_csv(os.path.join(base_path, "Products.csv"))

### CRM and Search Modules

In [18]:
# actual CRM module/assistant to add

# crm = CRMAssistant(
#     accounts_df=accounts,
#     pipeline_df=pipeline,
#     teams_df=teams,
#     products_df=products
# )

In [ ]:
# Dummy CRM class (CRM for Testing)
# Replace this code block with actual CRM module
class DummyCRM:
    def process_query(self, query, context=None):
        # Simple placeholder logic
        response = f"Received your query: '{query}'"
        if context:
            response += f" with context: {context}"
        return response

# Create the crm object
crm = DummyCRM()

### Data Visualization & Analytics

In [27]:
# Task 3c: Data Visualization & Analytics (Starter)
def generate_chart(account=None, agent=None, timeframe=None):
    fig, ax = plt.subplots(figsize=(10, 11))
    df = pipeline.copy()

    if account:
        df = df[df["account"] == account]
    if agent:
        df = df[df["sales_agent"] == agent]
    if timeframe:
        df = df[df["engage_date"] >= timeframe]

    summary = df.groupby("deal_stage")["close_value"].sum()
    summary.plot(kind="bar", ax=ax)
    ax.set_title("Deal Value by Stage")
    ax.set_ylabel("Close Value ($)")
    ax.set_xlabel("Deal Stage")

    return fig

### Chatbot Implementation

In [22]:
# CRM chatbot function
def crm_chatbot(user_query, account_filter, agent_filter, timeframe_filter):
    context = {}
    if account_filter and account_filter != "None":
        row = accounts[accounts["account"] == account_filter].iloc[0].to_dict()
        context.update(row)
    if agent_filter and agent_filter != "None":
        context["sales_agent"] = agent_filter
    if timeframe_filter:
        context["timeframe"] = timeframe_filter

    llm_response = crm.process_query(user_query, context=context)
    chart = generate_chart(
        account_filter if account_filter != "None" else None,
        agent_filter if agent_filter != "None" else None,
        timeframe_filter
    )
    return llm_response, chart

In [23]:
# wrapper to handle "None" dropdowns
account_choices = ["None"] + sorted(accounts["account"].unique())
agent_choices = ["None"] + sorted(pipeline["sales_agent"].unique())

def crm_chatbot_wrapper(user_query, account_filter, agent_filter, timeframe_filter):
    if account_filter == "None":
        account_filter = None
    if agent_filter == "None":
        agent_filter = None
    return crm_chatbot(user_query, account_filter, agent_filter, timeframe_filter)

### Final Gradio Interface

In [30]:
# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("<h1 style='text-align: center; font-size: 40px;'>CRM AI Assistant</h1>")
    gr.Markdown("<h2 style='text-align: center; font-size: 20px;'>Salesforce Team2B</h2>")
    gr.Markdown("<p style='text-align: center; font-size: 15px;'>Our CRM AI Assistant is your go-to buddy for valuable insights. It’ll help you get any answer you need!</p>")
    gr.Markdown("<p style='text-align: center; font-size: 16px; font-weight: bold;'>Ask Questions. Get Insights. See Charts.</p>")

    with gr.Row():
        user_query = gr.Textbox(label="Your Question")
    
    with gr.Row():
        account_filter = gr.Dropdown(
            choices=account_choices,
            label="Filter by Account (optional)",
            value="None"
        )
        agent_filter = gr.Dropdown(
            choices=agent_choices,
            label="Filter by Sales Agent (optional)",
            value="None"
        )
        timeframe_filter = gr.Textbox(
            label="Filter by Date (YYYY-MM-DD)",
            placeholder="2025-01-01"
        )

    with gr.Row():
        submit_btn = gr.Button("Generate Insight", elem_id="submit-btn")

    gr.HTML("""
    <style>
    #submit-btn {
        background-color: #1E90FF; /* Dodger Blue */
        color: white;
        font-weight: bold;
        border-radius: 8px;
        padding: 10px 20px;
        font-size: 16px;
    }
    #submit-btn:hover {
        background-color: #104E8B; /* Darker blue on hover */
    }
    </style>
    """)

    with gr.Row():
        llm_output = gr.Textbox(label="Insight", lines=10)
    
    chart_output = gr.Plot(label="Data Visualization")

    submit_btn.click(
        crm_chatbot_wrapper,
        inputs=[user_query, account_filter, agent_filter, timeframe_filter],
        outputs=[llm_output, chart_output]
    )

demo.launch()

* Running on local URL:  http://127.0.0.1:7929
* To create a public link, set `share=True` in `launch()`.
